In [ ]:
###########Script to query SQlite database.###########

In [324]:
#Script pour exécuter des commandes shell et créer des csv à partir de requêtes SQL dans un conteneur Docker SQLite
import subprocess

def run_command(command, shell=False):
    """Helper to run shell or list-based commands with error handling."""
    try:
        result = subprocess.run(command, shell=shell, check=True, capture_output=True, text=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error running command: {command}")
        print(e.stderr)



In [325]:
#Start the container using the shell script
print("🚀 Starting Docker container with start_sqlite.sh...")
COMPOSE_FILE="../docker/docker-compose-sqlite.yml"
run_command(["docker", "compose", "-f", COMPOSE_FILE, "up", "-d", "--build"])


🚀 Starting Docker container with start_sqlite.sh...
#1 [internal] load local bake definitions
#1 reading from stdin 572B done
#1 DONE 0.0s

#2 [internal] load build definition from Dockerfile.sqlite
#2 transferring dockerfile: 185B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/alpine:3.19
#3 DONE 0.3s

#4 [internal] load .dockerignore
#4 transferring context: 277B done
#4 DONE 0.0s

#5 [1/3] FROM docker.io/library/alpine:3.19@sha256:6baf43584bcb78f2e5847d1de515f23499913ac9f12bdf834811a3145eb11ca1
#5 resolve docker.io/library/alpine:3.19@sha256:6baf43584bcb78f2e5847d1de515f23499913ac9f12bdf834811a3145eb11ca1 0.0s done
#5 DONE 0.0s

#6 [2/3] RUN apk add --no-cache sqlite bash
#6 CACHED

#7 [3/3] WORKDIR /data
#7 CACHED

#8 exporting to image
#8 exporting layers done
#8 exporting manifest sha256:c62068895fb5c5a190e6f8ddea60b4068e0cf398f6cbcf7d6a453e973dd3226b done
#8 exporting config sha256:670c789e61ba0ac6b4bdef63ccc33a321b46d17954b04c06fb4c5c48e40a4a06 done
#8 exp

In [ ]:
#Copy the SQLite DB into the container
print("📁 Copying newIMDB.db into docker container...")
run_command(["docker", "cp", "../data/db/newIMDB.db", "moviesdb_sqlite:/data/newIMDB.db"])



In [327]:
#copy the SQL file into the container
print("📁 Copying marvel_movies.sql into docker container...")
run_command(["docker", "cp", "../utils/marvel_movies.sql", "moviesdb_sqlite:/data/marvel_movies.sql"])

📁 Copying marvel_movies.sql into docker container...



In [328]:
#Run the SQL file using sqlite3 inside the container
print("📄 Executing marvel_queries.sql inside the container...")
run_command([
    "docker", "exec", "-i", "moviesdb_sqlite",
    "sqlite3", "/data/newIMDB.db", ".read /data/marvel_movies.sql"
])

📄 Executing marvel_queries.sql inside the container...



In [330]:
#copy csv file created by marvel_queries.sql from container to local
run_command([
    "docker", "cp", "moviesdb_sqlite:/data/marvel_movies.csv", "../data/tests/marvel_movies.csv"
])

In [331]:
#read csv into pandas dataframe
import pandas as pd
marvel_movies_df = pd.read_csv("../data/tests/marvel_movies.csv")
print("Returned rows:", len(marvel_movies_df))
marvel_movies_df.head()

EmptyDataError: No columns to parse from file

In [ ]:
marvel_movies_df.to_csv("../data/tests/marvel_movies_cleaned.csv", index=False)


In [ ]:
#copy the SQL file into the container
# print("📁 Copying marvel_characters.sql into docker container...")
# run_command(["docker", "cp", "../utils/marvel_characters.sql", "moviesdb_sqlite:/data/marvel_characters.sql"])

In [ ]:
# #Run the SQL file using sqlite3 inside the container
# print("📄 Executing marvel_characters.sql inside the container...")

# run_command([
#     "docker", "exec", "-i", "moviesdb_sqlite",
#     "sh", "-c", "sqlite3 /data/newIMDB.db < /data/marvel_characters.sql"
# ])  

In [ ]:
# run_command([
#     "docker", "cp", "moviesdb_sqlite:/data/marvel_characters.csv", "../data/tests/marvel_characters.csv"
# ])

In [ ]:
# # read csv into pandas dataframe
# import pandas as pd
# marvel_characters_df = pd.read_csv("../data/tests/marvel_characters.csv")

# # character_name is formatted as a string representation of a list, convert it back to list and get first element
# import ast

# marvel_characters_df['character_name'] = marvel_characters_df['character_name'].str.replace(r"\s*\([^)]*\)", "", regex=True)

# marvel_characters_df['character_name'] = marvel_characters_df['character_name'].apply(
#     lambda x: ast.literal_eval(x)[0] if pd.notnull(x) else None
# )

# marvel_characters_df.head()

In [ ]:
# marvel_characters_df.to_csv("../data/tests/marvel_characters_cleaned.csv", index=False)


In [ ]:
#Stop and remove the container
print("🚀 Stopping Docker container with stop_sqlite.sh...")
COMPOSE_FILE="../docker/docker-compose-sqlite.yml"
CONTAINER_NAME="moviesdb_sqlite"

run_command(["docker", "stop", CONTAINER_NAME])
run_command(["docker", "compose", "-f", COMPOSE_FILE, "down"])